## Importing modules and dataset

In [2]:
import numpy as np
import pandas as pd
import random
import math
import requests
import io
url = "https://raw.githubusercontent.com/Sankalp2002/IML_datasets/main/hp_dataset.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df=df.loc[:,"price":"bathrms":1]
print(df)
#dividing into training and testing parts
df_test=df.iloc[383:546]
df_train=df.iloc[0:383]

        price  lotsize  bedrooms  bathrms
0     42000.0     5850         3        1
1     38500.0     4000         2        1
2     49500.0     3060         3        1
3     60500.0     6650         3        1
4     61000.0     6360         2        1
..        ...      ...       ...      ...
541   91500.0     4800         3        2
542   94000.0     6000         3        2
543  103000.0     6000         3        2
544  105000.0     6000         3        2
545  105000.0     6000         3        1

[546 rows x 4 columns]


### Scale function takes list and returns the mean scaled list

In [3]:
def scale(A):
    _mean=np.mean(A)
    _max=max(A)
    _min=min(A)
    scaled=[((i - _mean) / (_max - _min)) for i in A]
    return scaled

### Creating a scaled list for combined features and a list for price for both training and testing parts.

In [59]:
y = (list(df_train.iloc[:,0]))
x0 = [1]*len(y)
x1 = scale(list(df_train.iloc[:,1]))
x2 = scale(list(df_train.iloc[:,2]))
x3 = scale(list(df_train.iloc[:,3]))
scaled_FTrain=list(zip(x0,x1,x2,x3))
scaled_PTrain=y
y = (list(df_test.iloc[:,0]))
x0 = [1]*len(y)
x1 = scale(list(df_test.iloc[:,1]))
x2 = scale(list(df_test.iloc[:,2]))
x3 = scale(list(df_test.iloc[:,3]))
scaled_PTest =y
scaled_FTest=list(zip(x0,x1,x2,x3))
# y = (list(df_train.iloc[:,0]))
# x0 = [1]*len(y)
# x1 = (list(df_train.iloc[:,1]))
# x2 = (list(df_train.iloc[:,2]))
# x3 = (list(df_train.iloc[:,3]))
# scaled_FTrain=list(zip(x0,x1,x2,x3))
# scaled_PTrain=y
# y = (list(df_test.iloc[:,0]))
# x0 = [1]*len(y)
# x1 = (list(df_test.iloc[:,1]))
# x2 = (list(df_test.iloc[:,2]))
# x3 = (list(df_test.iloc[:,3]))
# scaled_PTest =y
# scaled_FTest=list(zip(x0,x1,x2,x3))

# Q6 C
Design Predictor using Batch Gradient Descent Algorithm, Stochastic Gradient Algorithm and mini batch Gradient Descent algorithms (determining minibatch size is your choice- here it could be 10, 20, 30 etc.) with and without regularization and compare their performances in terms of % error in prediction.

## Creating separate slope functions for batch, stochastic and mini-batch gradient descent.

### Slopebgd calculates slope for all samples for the gradient descent part.
### Slopesgd calculates slope for a single sample at a given index. 
### Slopembgd calculates slope for a batch of samples starting at a given index till a given batch size.

In [5]:
def Slopebgd(C, F, P, ind):
    Error = 0
    for i in range(len(F)):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopembgd(C, F, P, ind,bsize,k):
    Error = 0
    for i in range(k,k+bsize):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopesgd(C,F,P,ind,i):
    h = 0
    for j in range(len(C)):
        h = h + C[j] * F[i][j]
    return (h - P[i]) * F[i][ind]

## Creating an error function which returns mean percentage error for given coefficients and feature, price testing list.

In [6]:
def CalcError(F2,P2,Coeff):
    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error

## Functions for Batch Gradient Descent with and without regularization.
### Takes training and testing features and price lists, learning rate, no. of iterations as input and returns error%.
### Regularized BGD takes lambda as input also.

In [95]:
def bgd(F1,P1,F2,P2,rate,iters):
    Coeff = [0, 0, 0, 0]
    for i in range(iters):#50000
        Temp= Coeff
        for j in range(len(Coeff)):
            Temp[j] = Temp[j] - ((rate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)))
        Coeff = Temp
    #print(Coeff)
    return CalcError(F2,P2,Coeff)

def bgd_reg(F1,P1,F2,P2,rate,iters,lam):
    Coeff = [0, 0, 0, 0]
    for i in range(iters):#50000
        Temp = Coeff
        for j in range(len(Coeff)):
            Temp[j] = Temp[j] - ((rate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)+lam*Temp[j]))
        Coeff = Temp
    return CalcError(F2,P2,Coeff)

## Functions for Stochastic Gradient Descent with and without regularization.
### Takes training and testing features and price lists, learning rate, no. of iterations as input and returns error%.
### Regularized SGD takes lambda as input also.

In [94]:
def sgd(F1,P1,F2,P2,rate,iters):
    Coeff = [0,0,0,0]
    for iter in range(iters):#1000000
        i=random.randint(0,382)# sample is randomly selected
        Temp = Coeff
        for j in range(4):
            Temp[j] = Temp[j] - ((rate/len(F1)) * (Slopesgd(Coeff, F1, P1, j,i)))
        Coeff = Temp
    return CalcError(F2,P2,Coeff)

def sgd_reg(F1,P1,F2,P2,rate,iters,lam):
    Coeff = [0,0,0,0]
    for iter in range(iters):#1000000
        i=random.randint(0,382)
        Temp = Coeff
        for j in range(4):
            Temp[j] = Temp[j] - ((rate/len(F1)) * (Slopesgd(Coeff, F1, P1, j,i)+lam*Temp[j]))
        Coeff = Temp
    return CalcError(F2,P2,Coeff)

## Functions for Mini-Batch Gradient Descent with and without regularization.
### Takes training and testing features and price lists, learning rate,batch size, no. of iterations as input and returns error%.
### Regularized MBGD takes lambda as input also.

In [156]:
def mbgd(F1,P1,F2,P2,rate,bsize,iters):
    Coeff = [0,0,0,0]
    NoOfBatches = math.floor(len(P1) / bsize)
    for e in range(iters):
        batch = random.randrange(0,NoOfBatches)
        Temp = Coeff
        for j in range(len(Coeff)):
            Temp[j] = Temp[j] - ((rate / len(F1)) * (Slopembgd(Coeff, F1, P1, j,bsize,bsize*batch)))
        Coeff = Temp
    return CalcError(F2,P2,Coeff)

def mbgd_reg(F1,P1,F2,P2,rate,bsize,iters,lam):
    Coeff = [0,0,0,0]
    NoOfBatches = math.floor(len(P1) / bsize)
    for e in range(iters):
        batch = random.randrange(0,NoOfBatches)#starting point of batch is randomly selected
        Temp = Coeff
        for j in range(len(Coeff)):
            Temp[j] = Temp[j] - ((rate / len(F1)) * (Slopembgd(Coeff, F1, P1, j,bsize,bsize*batch)+lam*Temp[j]))
        Coeff = Temp  
    return CalcError(F2,P2,Coeff)

In [161]:


print(bgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001,8100))

    
print(sgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.0005,10000000))


print(mbgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001,10,500000))

[64134.9530024952, 42125.48662362529, 24281.433569612625, 40089.23105777997]
25.879592603996134


In [163]:


print(bgd_reg(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001,8100,1))



# print(sgd_reg(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.003,1))



# print(mbgd_reg(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001,10,500000,1))

19.869640072997548


# Q6 D

In [ ]:
def bgd(F,P,rate,iters):
    LearningRate = rate
    Coeff = [0, 0, 0, 0]
    for i in range(iters):#50000
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)))
        Coeff = TempCoeff.copy()
    return CalcError(F2,P2,Coeff)